# Disney World Reviews Analysis


Data Source: https://www.kaggle.com/arushchillar/disneyland-reviews <br>


Disneyland Reviews dataset consists of reviews and ratings of 3 Disneyland location (namely California, Paris & Hongkong), posted by visitors on TripAdvisor.<br>

Number of reviews: 42,000<br>
Timespan: Oct 2010 - May 2019<br>
Number of Attributes/Columns in data: 6 

Attribute Information:

1. Review_ID: unique id given to each review
2. Rating: ranging from 1 (unsatisfied) to 5 (satisfied)
3. Year_Month: when the reviewer visited the theme park
4. Reviewer_Location: country of origin of visitor
5. Review_Text: comments made by visitor
6. Disneyland_Branch: location of Disneyland Park


#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.


# [1]. Reading Data

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


# using the SQLite Table to read data.
import sqlite3

import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#from gensim.models import Word2Vec
#from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [2]:
con = sqlite3.connect('disneyReviews.db') 

In [3]:
# Selecting only positive and negative reviews i.e. 
# Rating=3 will be ignored as this rating is neutral (ie neither positive or negative)
# SELECT * FROM Reviews WHERE Rating != 3 LIMIT 20000, will give top 20000 data points
# We can change the number to any other number based on your computing power

filtered_df = pd.read_sql_query(""" SELECT * FROM DisneylandReviews WHERE Rating!=3 LIMIT 20000 """, con)

# filtered_df = pd.read_sql_query(""" SELECT * FROM DisneylandReviews WHERE Rating!=3 LIMIT 20000 """, con)

In [4]:
# Convert reviews with Rating>3 a positive rating, and reviews with a Rating<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

In [6]:
#changing reviews with Rating less than 3 to be positive and Rating greater than 3 to be negative
actualScore = filtered_df['Rating']
positiveNegative = actualScore.map(partition) 
filtered_df['Rating'] = positiveNegative
print("Number of data points in our data", filtered_df.shape)
filtered_df.head(3)

Number of data points in our data (20000, 6)


,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,1,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,1,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,1,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong


In [14]:
display_df = pd.read_sql_query("""
SELECT Review_ID,Rating, Year_Month, Reviewer_Location, Review_Text, Branch, COUNT(*)
FROM DisneylandReviews
GROUP BY Review_Text
HAVING COUNT(*)>1
""", con)

In [15]:
print(display_df.shape)
display_df.head()

(24, 7)


,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch,COUNT(*)
0,198850214,4,2014-3,United States,3 day Military Hopper Pass; best deal around. ...,Disneyland_California,2
1,606997669,5,2018-8,France,ActiveX VT ERROR:,Disneyland_Paris,2
2,166784597,4,2013-5,United States,Disneyland we love it! The service is incomp...,Disneyland_California,2
3,226905150,5,2014-5,United States,Disneyland Paris is different then other Disne...,Disneyland_Paris,2
4,239871388,4,2014-10,Canada,"Disneyland, Hong Kong Disneyland (Hong Kong) i...",Disneyland_HongKong,2


In [27]:
display_df[display_df['Review_ID']==198850214]

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch,COUNT(*)
0,198850214,4,2014-3,United States,3 day Military Hopper Pass; best deal around. ...,Disneyland_California,2


In [29]:
display_df['COUNT(*)'].sum()

48

In [28]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

#  Exploratory Data Analysis

## [2] Data Preprocessing

It is observed (as shown in the table below) that the reviews data had significant (~1%) duplicate entries. Hence, it is necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [30]:
display_df.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch,COUNT(*)
0,198850214,4,2014-3,United States,3 day Military Hopper Pass; best deal around. ...,Disneyland_California,2
1,606997669,5,2018-8,France,ActiveX VT ERROR:,Disneyland_Paris,2
2,166784597,4,2013-5,United States,Disneyland we love it! The service is incomp...,Disneyland_California,2
3,226905150,5,2014-5,United States,Disneyland Paris is different then other Disne...,Disneyland_Paris,2
4,239871388,4,2014-10,Canada,"Disneyland, Hong Kong Disneyland (Hong Kong) i...",Disneyland_HongKong,2


As displayed above the same Reviewer_ID has multiple reviews of the with the same values for Rating, Year_MOnth, Reviewer_LOcation, Branch and Review_Text. On further analysis it was found that <br>
<br> 
Review_text is exactly same for the duplicate reviews.

It was inferred after analysis that reviews with same parameters. Therefore, it is imperative to reduce redundancy. Decided to eliminate the rows having same exact parameters to avoid bias.<br>

The method used for the same was that we first sort the data according to Review_ID and then just keep the first similar Review_Text and drop redundant records from the dataframe. for eg. in the above just the review for Review_ID=198850214, first occurence remains. This method ensures that there is only one representative for each Review/Rating and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [31]:
#Sorting data according to Review_ID in ascending order
sorted_df=filtered_df.sort_values('Review_ID', axis=0, ascending=True, inplace=False, kind='mergesort', na_position='last')

In [32]:
#Deduplication of entries
final_df=sorted_df.drop_duplicates(subset={"Review_ID","Year_Month","Reviewer_Location","Review_Text","Branch"}, keep='first', inplace=False)
final_df.shape

(19993, 6)

In [34]:
#Calculate % of data still remains post deduplication
(final_df['Review_ID'].size*1.0)/(final_df['Review_ID'].size*1.0)*100

100.0

In [38]:
#Get the dataframe shape, before starting the Text-preprocessing phase
print(final_df.shape)

#Check number of positive and negative reviews are present in our dataset?
final_df['Rating'].value_counts()


(19993, 6)


1    18586
0     1407
Name: Rating, dtype: int64

Observation : Deduplication (Perhaps, Original) dataset is imbalanced with more than ~92% of records are positive reviews

# [3].  Text Preprocessing.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews